# Diagnose raw .dat file


- shape
- data types
- nulls 
- flow distinct values
- value and quantity ranges
- sample rows
- zero and negative values

*I choose Duckdb because it's faster (reads the file directly, no parsing boilerplate)  and it works with SQL*

In [ ]:
import duckdb
from pathlib import Path

BASE_PATH = Path.cwd().parent / "data" / "raw" / "comext_products"
FILE_PATH =  BASE_PATH / "2002-12" / "full_v2_200212.dat"

rel = duckdb.read_csv(FILE_PATH)

# Shape
rows = duckdb.sql("SELECT COUNT(*) AS rows FROM rel")
col_num = len(rel.columns)
print(f"\nRows: \n{rows} \nNumber of columns: \n{col_num}\n ")

# Data Types
col_names = rel.columns
data_types = rel.dtypes
for col, dtype in zip(col_names, data_types):
    print(f"{col}: {dtype}")

In [ ]:
# Nulls (only the 7 columns we keep)
duckdb.sql("""
    SELECT
        COUNT(*) - COUNT(REPORTER)    AS reporter_nulls,
        COUNT(*) - COUNT(PARTNER)     AS partner_nulls,
        COUNT(*) - COUNT(PRODUCT_NC)  AS product_nc_nulls,
        COUNT(*) - COUNT(FLOW)        AS flow_nulls,
        COUNT(*) - COUNT(PERIOD)      AS period_nulls,
        COUNT(*) - COUNT(VALUE_EUR)   AS value_eur_nulls,
        COUNT(*) - COUNT(QUANTITY_KG) AS quantity_kg_nulls
    FROM rel
""")

In [ ]:
# FLOW distinct values
print(duckdb.sql("SELECT FLOW, COUNT(*) AS count FROM rel GROUP BY FLOW ORDER BY FLOW"))

# VALUE_EUR and QUANTITY_KG ranges
duckdb.sql("""
    SELECT
        MIN(VALUE_EUR)    AS value_eur_min,
        MAX(VALUE_EUR)    AS value_eur_max,
        MIN(QUANTITY_KG)  AS quantity_kg_min,
        MAX(QUANTITY_KG)  AS quantity_kg_max
    FROM rel
""")

In [ ]:
# Sample rows (only the 7 columns we keep)
duckdb.sql("""
    SELECT REPORTER, PARTNER, PRODUCT_NC, FLOW, PERIOD, VALUE_EUR, QUANTITY_KG
    FROM rel
    LIMIT 10
""")

In [ ]:
# Zero and negative values
duckdb.sql("""
    SELECT
        COUNT(*) FILTER (WHERE VALUE_EUR = 0)    AS value_eur_zeros,
        COUNT(*) FILTER (WHERE QUANTITY_KG = 0)  AS quantity_kg_zeros,
        COUNT(*) FILTER (WHERE VALUE_EUR < 0)    AS value_eur_negatives,
        COUNT(*) FILTER (WHERE QUANTITY_KG < 0)  AS quantity_kg_negatives
    FROM rel
""")